## Setup for Websocket Connection (Binance & Coinbase Pro)

In [1]:
import asyncio
from datetime import datetime, timedelta
import sys
#from binance.client import AsyncClient
# from binance import BinanceSocketManager, ThreadedWebsocketManager
import configparser
import utils
from websocket import create_connection
import websockets

import nest_asyncio
nest_asyncio.apply()

# BINANCE_SYMBOL = 'btcusdt'
# COINBASE_SYMBOL = 'BTC-USD'
BINANCE_SUBSCRIBE = '{ "method": "SUBSCRIBE", "params": [ "btcusdt@kline_1M"], "id": 1 }'
COINBASE_SUBSCRIBE = '{ "type": "subscribe", "product_ids": ["BTC-USD"], "channels": [ "ticker"] }'

config = configparser.ConfigParser()
config.read('../config.ini')

B_WSS = config['BINANCE']['WS_URL']
C_WSS = config['COINBASE']['WS_URL']

## Websocket Testing for Binance (1)

In [ ]:
ws = create_connection(B_WSS)
ws.send(BINANCE_SUBSCRIBE)

# Infinite loop waiting for WebSocket data
while True:
    print(ws.recv())



## Websocket Testing for Binance (2)

In [ ]:
connection = websockets.connect(uri=B_WSS)

async with connection as websocket:
    # Sends a message.
    await websocket.send(BINANCE_SUBSCRIBE)

    # Receives the replies.
    async for message in websocket:
        print(message)
        
    # Closes the connection.
    await websocket.close()


## Websocket Testing for Coinbase Pro (1)

In [ ]:
# Connect to WebSocket API and subscribe to trade feed for XBT/USD and XRP/USD
connection = websockets.connect(uri=C_WSS)

async with connection as websocket:
    # Sends a message.
    await websocket.send(COINBASE_SUBSCRIBE)

    # Receives the replies.
    async for message in websocket:
        print(message)
        
    # Closes the connection.
    await websocket.close()

## Websocket Testing for Coinbase Pro (2)

In [ ]:
async def work(uri, subscribe):

    connection = websockets.connect(uri=uri)

    async with connection as websocket:
        # Sends a message.
        # await websocket.send('{ "type": "subscribe", "product_ids": ["BTC-USD"], "channels": [ "ticker"] }')
        await websocket.send(subscribe)

        # Receives the replies.
        async for message in websocket:
            print(message)
        
        # Closes the connection.
        await websocket.close()


loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(work(C_WSS, COINBASE_SUBSCRIBE))
finally:
    loop.close()

## Multi Websocket Connection Testing for Binance & Coinbase Pro

In [2]:
urls = [B_WSS, C_WSS]
sub_texts = [BINANCE_SUBSCRIBE, COINBASE_SUBSCRIBE]


async def socket_communication(url, sub_text):
    connection = websockets.connect(uri=url)

    async with connection as websocket:
        # Sends a message.
        await websocket.send(sub_text)
        print(f'connected to {url}')

        # Receives the replies.
        async for message in websocket:
            print(url, message)
            await asyncio.sleep(0)
        
        # Closes the connection.
        await websocket.close()


async def main():
    tasks = []

    for i in range(2):
        tasks.append(socket_communication(urls[i],sub_texts[i]))

    # print(tasks)
    await asyncio.gather(*tasks)

asyncio.run(main())

[<coroutine object socket_communication at 0x7f621f7ea340>, <coroutine object socket_communication at 0x7f621f7ea3c0>]
connected wss://fstream3.binance.com/ws - ['{ "method": "SUBSCRIBE", "params": [ "btcusdt@kline_1M"], "id": 1 }', '{ "type": "subscribe", "product_ids": ["BTC-USD"], "channels": [ "ticker"] }']
wss://fstream3.binance.com/ws {"id":1,"result":null}
wss://fstream3.binance.com/ws {"e":"kline","E":1621558592681,"s":"BTCUSDT","k":{"t":1619827200000,"T":1622505599999,"s":"BTCUSDT","i":"1M","f":745987808,"L":843568511,"o":"57688.29","c":"41649.99","h":"59654.00","l":"28688.00","v":"9162628.663","n":97513080,"x":false,"q":"446520599719.57567","V":"4506277.465","Q":"219460791226.18320","B":"0"}}
wss://fstream3.binance.com/ws {"e":"kline","E":1621558593043,"s":"BTCUSDT","k":{"t":1619827200000,"T":1622505599999,"s":"BTCUSDT","i":"1M","f":745987808,"L":843568534,"o":"57688.29","c":"41630.02","h":"59654.00","l":"28688.00","v":"9162630.074","n":97513103,"x":false,"q":"446520658477.71

KeyboardInterrupt: 